# Teoria de Números Computacionais
## Trabalho 3: Ataque cíclico ao RSA

#### Hugo Sousa (a76257 - LCC)
#### Matias Capitão (a82726 - LCC) 
#### Rafael Antunes (a77457 - LCC)

### Ataque cíclico ao RSA


Tomemos o sistema de cifração RSA. Sejam **p e q** dois primos, $n = pq$ e **e** tal que $0 < e < ϕ(n)$ e $mdc(e, ϕ(n)) = 1$ o expoente de cifração da chave público, e por fim __d__ tal que $e*d ≡ 1 (mod ϕ(n))$ o expoente de decifração da chave privada. e $M∈Z_n$ uma mensagem.

Seja $C$ a mensagem cifrada tal que $C = M^e mod n$, e  seja **k** um inteiro positivo tal que $C^{e^k} mod n = C$, como a encriptação é uma permutação no espaço da mensagem ou seja é uma permutação em $\{0, 1, ..., n-1\}$ temos que __k__ existe e pela mesma razão $C^{e^{k-1}} mod n = M$. esta propriedade leva à formação do ataque cíclico à encriptação RSA.

### Formulação do Ataque

Vamos supor que Alice manda uma mensagem a Bob então Bob irá receber a mensagem cifrada $C$ tal que $C = M^e mod n$.

Vamos supor agora que Chuck intercepta a mensagem cifrada $C$, Chuck tem a mensagem cifrada e tem acesso à chave pública de bob, entao para descobrir a mensagem pode efetuar o seguinte algoritmo:



<center> Chuck computa $C^{e^1} mod$ n , $C^{e^2} mod$ n, ..., até que para algum p:  $C^{e^p} mod n = M$ </center>
<center> Ora isto significa então que $C^{e^{p+1}} mod n = C$ </center>


#### Nota:


Note-se que Chuck não precisa de ter acesso à mensagem original embora a formulação do ataque assim o faça parecer, pois pela propriedade que vimos em cima, temos que, existe um **k** tal que $C^{e^k} mod n = C mod n$ então


<center> $C^{e^{k-1}} mod n= C^{e^{k}*e^{-1}} mod n= (C^{e^k})^{e^{-1}} mod n= C^{e^{-1}} mod n$</center>

logo, pela definição de chave privada temos que $d = e^{-1} (mod ϕ(n))$ donde resulta que:
<center> $C^{e^{k-1}} = C^d = M(mod n)$ </center>

In [1]:
#Classe que define um sistema RSA com funções de encriptação, decriptação
class RSA_cipher:

    def __init__(self):
        n,e,d = RSA_cipher.gen_keys()
        self.n = n # p*q
        self.public = e # exponent of public key
        self.private = d # exponent of private key

    def publish(self):
        return(self.n, self.public)

    #Cipher a message to send using the public key of the receiver
    def cipher(self, msg, public_key):
        return power_mod(msg, public_key[1], public_key[0])


    #Decipher a message using private key
    def decipher(self, msg):
        return power_mod(msg, self.private, self.n)

    #Function to generate the public and private keys
    @staticmethod
    def gen_keys(nbits = 16):
        p = random_prime(2^(nbits//2), 2^(nbits//2-2))
        q = random_prime(2^(nbits//2 +1), 2^(nbits//2 -1))

        while p == q:
            p = random_prime(2^(nbits//2), 2^(nbits//2-2))
            q = random_prime(2^(nbits//2 +1), 2^(nbits//2 -1))

        n = p*q
        m = (p-1)*(q-1)
        e = randint(2, m-1)
        while gcd(e, m) != 1:
            e = randint(2,m-1)

        d = power_mod(e, -1, m)

        return n,e,d

### Exemplo

In [2]:
#Alice cria o seu sistema de encriptação RSA e publica a sua chava publica
Alice = RSA_cipher()
public_alice = Alice.publish()
public_alice

(28243, 24881)

In [3]:
#Bob quer mandar uma mensagem a Alice
bob_msg = 123

#Bob cifra a sua mensagem com a chave publica de alice
Bob = RSA_cipher()
enc_msg = Bob.cipher(bob_msg, public_alice)
enc_msg

9700

In [4]:
#Chuck intercepta a mensagem encriptada e vai proceder ao ataque

#função que vai cifrar em ciclo a mensagem com a chave publica de ALice até obter a mensagem cifrada original retornando 
#o número de vezes que foi necessário cifrar a mensagem para obter a cifração original
def cyclic_attack(msg, enc_msg, pk):
    i = 1
    while(power_mod(enc_msg, pow(pk[1],i) , pk[0]) != enc_msg):
        i += 1
    return i

k = cyclic_attack(bob_msg, enc_msg, public_alice)
k

6

In [5]:
m1 = power_mod(enc_msg, pow(public_alice[1], (k-1)), public_alice[0])
m1

123

In [6]:
#Note-se então que este processo dá o mesmo resultado que Alice decifrar a mensagem com a sua chave privada
m2 = Alice.decipher(enc_msg)

m1 == m2

True

#### Nota final:

É de notar que este tipo de ataques é inefectivo num cenário real visto que ele é análogo a factorizar n.

### Bibliografia

A. Menezes, P. van Oorschot and S. Vanstone. - _Handbook of Applied Cryptography_

<https://crypto.stackexchange.com/questions/1572/cycle-attack-on-rsa>